In [1]:
import cv2
import numpy as np

In [2]:
from ultralytics import YOLO
# Kullanacağımız modeli yüklüyoruz
model = YOLO("yolov8l.pt")

Bu fonksiyonu kullanarak mouse ile ekranda bölge (dikdörtgen) çizebiliyoruz.

In [3]:
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, frame

    # Mouse'a basıldığında
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix, iy = x, y

    # Mouse hareket ederken ve basılıyken
    elif event == cv2.EVENT_MOUSEMOVE:
        if drawing:
            frame_copy = frame.copy()
            cv2.rectangle(frame_copy, (ix, iy), (x, y), (0, 255, 0), 2)
            cv2.imshow('Video', frame_copy)

    # Mouse bırakıldığında
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        rectangles.append((ix, iy, x, y)) 

In [6]:
# Çizilen dikdörtgenleri saklamak için bir liste
rectangles = []

# İlk durum ve koordinatları tutacak değişkenler
drawing = False
ix, iy = -1, -1


# Kullanacağımız videoyu yüklüyoruz
video = cv2.VideoCapture('people1.mp4')  
font = cv2.FONT_HERSHEY_SIMPLEX

# Pencere oluşturuyoruz 
cv2.namedWindow('Video')

# Bu fonksiyon mouse'a dokunulduğu zaman kendiliğinden çağrılıyor
# Çağıracağımız fonksiyonun adı draw_rectangle

cv2.setMouseCallback('Video', draw_rectangle)

while True:
    ret, frame = video.read()
    
    if not ret:
        break
        
        
    
    # görüntü boyutunu düşürmek isterseniz
    # frame=cv2.resize(frame,(0,0),fx=0.5,fy=0.5)
    
    # Resmi RGB uzayına çevirip nesne tespit eden modele veriyoruz
    imgs=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    results = model(imgs,verbose=False) 
    labels=results[0].names
    
    # Burada çizilen her bölge için dönecek for dögüsü
    for rect in rectangles:
        #Bölgedeki insan sayısını bununla tutuyoruz
        total=0
        
        # Bölgeyi ekranda çizdiriyoruz.
        cv2.rectangle(frame, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
        
        # Burada her bir nesne için dönecek for döngüsü
    
        for i in range(len(results[0].boxes)):
            x1,y1,x2,y2=results[0].boxes.xyxy[i]
            score=results[0].boxes.conf[i]
            label=results[0].boxes.cls[i]
            x1,y1,x2,y2,score,label=int(x1),int(y1),int(x2),int(y2),float(score),int(label)
            name=labels[label]
            
            
            # İnsan olmayan nesneleri işleme katmıyoruz
            if  name!='person': 
                continue
        
            # Nesnelerin orta noktası
            cx=int(x1/2+x2/2)
            cy=int(y1/2+y2/2)
            
            # Orta noktaları ekranda gösteriyoruz
            cv2.circle(frame,(cx,cy),5,(0,0,255),-1)
            

            
            # Mouse ile çizilen bölgeden koordinatları alıp uygun formata çeviriyoruz
            region1=np.array([(rect[0],rect[1]),(rect[2],rect[1]),(rect[2],rect[3]),(rect[0],rect[3])])
            region1 = region1.reshape((-1,1,2))
            
            # Nesnenin orta noktası bölgenin içinde mi diye bakıyoruz.
            inside_region1=cv2.pointPolygonTest(region1,(cx,cy),False)
            
            # Bölgeyi  ekranda sarı renk ile gösteriyoruz
            cv2.polylines(frame,[region1],True,(0,255,255),5)
            
        
            # Eğer nesne bölgenin içinde ise bu if True döner ve bölgedeki kişi sayısı 1 artar
            if inside_region1>0:
                
                total+=1
        
        # Bölgenin sol üst köşesinin 10 piksel üstünde bölgedeki kişi sayısını gösteriyoruz
        cv2.putText(frame, str(total),(rect[0],rect[1]-10), font, 2.1, (255,0,255), 3)

        
    
        

    cv2.imshow('Video', frame)
    

   
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


video.release()
cv2.destroyAllWindows()

Sadece bölgeleri çizmek isterseniz aşağıdaki kodu kullanabilirsiniz. 

In [7]:

rectangles = []


drawing = False
ix, iy = -1, -1

video = cv2.VideoCapture('people1.mp4')  
font = cv2.FONT_HERSHEY_SIMPLEX

# Pencere oluşturuyoruz 
cv2.namedWindow('Video')

cv2.setMouseCallback('Video', draw_rectangle)

while True:
    ret, frame = video.read()
    
    if not ret:
        break
        
        
    
    # görüntü boyutunu düşürmek isterseniz
    # frame=cv2.resize(frame,(0,0),fx=0.5,fy=0.5)

    
    # Burada çizilen her bölge için dönecek for dögüsü
    for rect in rectangles:
        
        cv2.rectangle(frame, (rect[0], rect[1]), (rect[2], rect[3]), (0, 255, 0), 2)
        

    cv2.imshow('Video', frame)
    

   
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


video.release()
cv2.destroyAllWindows()